# Анализ расчета градиента давления с использованием различных гидравлических корреляций

версия 0.1 от 5.5.20

Что то нарисовать получилось, но надо еще подумать, что получилось то

In [ ]:
# подготовимся к импорту API который находится в родительской папке относительно этой тетрадки
import sys
sys.path.insert(0,'..')
import pandas as pd
# импортируем api
import unifloc_vba_python_api.python_api as python_api 
# готовим объект с доступом к надстройке. При этом надстройка откроется и запустится excel 
UniflocVBA = python_api.API("../UniflocVBA_7.xlam")
print("import excel with unifloc vba done. version " + UniflocVBA.unf_version())

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]

## Описание функции  `MF_dpdl_atmm`
Расчет градиента давления может быть выполнен пользовательской функцией `MF_dpdl_atmm`
Функция рассчитывает перепад давления для заданных параметров (расходов жидкости и газа в рабочих условиях и свойтсвах флюидов). Также рассчитывается и перепад давления из за трения.

Расчет может быть проведен для нескольких корреляций, определяемых параметром `hcorr `. Возможные значения ниже. 
````python
    BeggsBrill = 0
    Ansari = 1
    Unified = 2
    Gray = 3
    HagedornBrown = 4
    SakharovMokhov = 5
    gas = 10
    Water = 11

````
Функция возвращает массив значений. Отдельные значения можно вытащить указав номер `param_out`
````python
param_out[0] = dPdLg_out_atmm * c_calibr_grav + dPdLf_out_atmm * c_calibr_fric  # общий градиент давления атм/м
param_out[1] =dPdLg_out_atmm * c_calibr_grav # градиент давления гравитационный атм/м
param_out[2] =dPdLf_out_atmm * c_calibr_fric # градиент давления трения атм/м
param_out[3] =dPdLa_out_atmm # градиент давления инерционный  атм/м
param_out[4] =Vsl_msec # приведенная скорость жидкости м/с
param_out[5] =Vsg_msec # приведенная скорость газа м/с
param_out[6] =Hl_out_fr # истинное содержание жидкости, liquid hold up
param_out[7] =fpat_out_num # номер режима потока
````

Режимы потока кодируются следующими номерами для корреляции Ансари
````python
fpat_out_num = 100 ' " liq" = liquid
fpat_out_num = 101 ' " gas" = gas
fpat_out_num = 105 ' "anul" = annular
fpat_out_num = 104 ' "dbub" = dispersed bubble
fpat_out_num = 103 ' "slug" = slug
fpat_out_num = 102 ' "bubl" = bubbly
fpat_out_num = 199
````

In [ ]:
def covert_vba_result(vba_result, option=0):
    if option == 0: #возвращаем только рассчитанные давление и температуру
        result = vba_result[0][0], vba_result[0][1]
    elif option == 1:
        result = pd.DataFrame(vba_result) #возвращает необработанный DataFrame
    elif option == 2:
        result = pd.DataFrame(vba_result) #возвращает необработанный DataFrame
        result.columns = result[result.index==2].iloc[0]
        result = result[result.index>2]
        result = result.set_index('hvert,m')
    return result    

In [ ]:
pvt_str_data = {"gamma_gas":0.6,
"gamma_oil":0.86,
"gamma_wat":1,
"rsb_m3m3":100,
"rp_m3m3":-1,
"pb_atma":-1,
"tres_C":90,
"bob_m3m3":-1,
"muob_cP":-1,
"PVTcorr":0,
"ksep_fr":0,
"p_ksep_atma":-1,
"t_ksep_C":-1,
"gas_only":False
               }

In [ ]:
pipeline_data = {"p_calc_from_atma": 20,
"t_calc_from_C": 20,
"t_val":40,
"h_list_m" : 2000,
"diam_list_mm" : 50,
"qliq_sm3day" : 100,
"fw_perc":25,
"q_gas_sm3day":0,
"str_PVT":UniflocVBA.PVT_encode_string(**pvt_str_data),
"calc_flow_direction":10,
"hydr_corr":0,
"temp_method":1,
"c_calibr":1,
"roughness_m":0.0001,
"out_curves":2,
"out_curves_num_points":10}

In [ ]:
vba_result = UniflocVBA.MF_p_pipeline_atma(**pipeline_data)
r = covert_vba_result(vba_result,2)
r

In [ ]:
for i in r.columns:
    print(i)

In [ ]:
def sense_profile(pipeline_data, pvt_str_data, parameter_sense, range_sense, pvt_edit=False, parameter_to_plot='p,atma'):
    fig = plt.figure()
    for i in range_sense:
        if pvt_edit:
            pvt_str_data[parameter_sense] = i
            pipeline_data['str_PVT'] = UniflocVBA.PVT_encode_string(**pvt_str_data)
        else:
            pipeline_data[parameter_sense] = i
        vba_result = UniflocVBA.MF_p_pipeline_atma(**pipeline_data)
        r = covert_vba_result(vba_result,2)
        plt.plot(r[parameter_to_plot], r.index * (-1), label = f"{parameter_sense} = {i}")
    plt.legend(loc='upper right')
    plt.xlabel(parameter_to_plot)
    plt.ylabel('Глубина, м')
    fig.suptitle(f"Распределение параметра {parameter_to_plot} в фонтанирующей скважине при различных {parameter_sense}", fontsize = 18)
    plt.show()

In [ ]:
sense_profile(pipeline_data.copy(), pvt_str_data.copy(), 'rsb_m3m3', [50, 60, 70], True, 'c_rhomix')

In [ ]:
sense_profile(pipeline_data.copy(), pvt_str_data.copy(), 'hydr_corr', [0,1,2,3,4,5], False)

In [ ]:
sense_profile(pipeline_data.copy(), pvt_str_data.copy(), 'qliq_sm3day', range(10,300,50), False)

In [ ]:
def sense_vlp(pipeline_data, pvt_str_data, parameter_sense, range_sense, pvt_edit=False, q_list=range(10,300,50)):
    fig = plt.figure()
    for i in range_sense:
        if pvt_edit:
            pvt_str_data[parameter_sense] = i
            pipeline_data['str_PVT'] = UniflocVBA.PVT_encode_string(**pvt_str_data)
        else:
            pipeline_data[parameter_sense] = i
        p_atm_list=[]
        for j in q_list:
            pipeline_data['qliq_sm3day'] = j
            vba_result = UniflocVBA.MF_p_pipeline_atma(**pipeline_data)
            p_atm, t_c = covert_vba_result(vba_result,0)
            p_atm_list.append(p_atm)
        plt.plot(q_list,p_atm_list, label = f"{parameter_sense} = {i}")
    plt.legend(loc='upper right')
    plt.xlabel('Дебит, м3/сут')
    plt.ylabel('Давление, атм')
    fig.suptitle("Анализ чувстительности VLP", fontsize = 18)
    plt.show()

In [ ]:
sense_vlp(pipeline_data.copy(), pvt_str_data.copy(), 'fw_perc', range(10,100,20), False, range(10,200,10))

In [ ]:
sense_vlp(pipeline_data.copy(), pvt_str_data.copy(), 'rsb_m3m3', range(10,311,50), True, range(10,200,10))